In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
# File Path
BASE_DIR = Path().resolve().parents[1]

# Load Arctic gridded temperature NetCDF file
arctic_path = BASE_DIR / "data" / "raw" / "uThickCDR_summaries_current_V2.0.nc"
ds = xr.open_dataset(arctic_path, decode_times= True)
print(ds)

<xarray.Dataset> Size: 77MB
Dimensions:         (time: 254102, Mean Max Min Sigma: 4)
Dimensions without coordinates: time, Mean Max Min Sigma
Data variables: (12/26)
    SOURCE          (time) <U12 12MB ...
    PLATFORM        (time) <U11 11MB ...
    INDEX           (time) int32 1MB ...
    YEAR            (time) int64 2MB ...
    MEAN_YDAY       (time) float32 1MB ...
    MIN_YDAY        (time) float32 1MB ...
    ...              ...
    UNCERT          (time, Mean Max Min Sigma) float32 4MB ...
    DRAFT           (time, Mean Max Min Sigma) float32 4MB ...
    THICK           (time, Mean Max Min Sigma) float32 4MB ...
    THICK_ICE_SNOW  (time, Mean Max Min Sigma) float32 4MB ...
    FREEBOARD       (time, Mean Max Min Sigma) float32 4MB ...
    FDATE           (time) float32 1MB ...
Attributes:
    Data Set:            Unified Sea Ice Thickness Data Record
    Producers:           Axel Schweiger, Polar Science Center, University of ...
    Info:                psc.apl.uw.edu/sea_

In [11]:
# 2. Extract relevant variables
# 'THICK' has shape (time, Mean Max Min Sigma), so we take [:, 0] for Mean
thick_mean = ds["THICK"].isel(**{"Mean Max Min Sigma": 0})
draft_mean = ds["DRAFT"].isel(**{"Mean Max Min Sigma": 0})
freeboard_mean = ds["FREEBOARD"].isel(**{"Mean Max Min Sigma": 0})
icecon_mean = ds["ICECON"].isel(**{"Mean Max Min Sigma": 0})

# 3. Create a DataFrame
df = pd.DataFrame({
    "year": ds["YEAR"].values,
    "month": ds["MONTH"].values,
    "lat": ds["LAT"].values,
    "lon": ds["LON"].values,
    "thick_m": thick_mean.values,
    "draft_m": draft_mean.values,
    "freeboard_m": freeboard_mean.values,
    "icecon": icecon_mean.values
})

# 4. Filter for Arctic latitudes (>= 65°N) and remove missing values
df = df[(df["lat"] >= 65) & (~df["thick_m"].isna())]

# 5. Annual mean values for all four variables
annual_means = df.groupby("year")[["thick_m", "draft_m", "freeboard_m", "icecon"]].mean().reset_index()

# 6. Save for later use
annual_means.to_csv(BASE_DIR / "data" / "processed" / "arctic_ice_thickness_annual.csv", index=False)

print(annual_means.head())

   year   thick_m  draft_m  freeboard_m  icecon
0  1947  0.535714      NaN          NaN     NaN
1  1948  1.258095      NaN          NaN     NaN
2  1949  1.152381      NaN          NaN     NaN
3  1950  1.386000      NaN          NaN     NaN
4  1951  1.201667      NaN          NaN     NaN
